In [50]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
destination = "../generated_data"
%matplotlib inline


In [51]:
# Given functions
def get_d(deg_true):
    """
    Inputs:
    deg_true: (int) degree of the polynomial g

    Returns:
    a: (np array of size (deg_true + 1)) coefficients of polynomial g
    """
    return 5 * np.random.randn(deg_true + 1)


def get_design_mat(x, deg):
    """
    Inputs:
    x: (np.array of size N)
    deg: (int) max degree used to generate the design matrix

    Returns:
    X: (np.array of size N x (deg_true + 1)) design matrix
    """
    X = np.array([x ** i for i in range(deg + 1)]).T
    return X


def draw_sample(deg_true, a, N):
    """
    Inputs:
    deg_true: (int) degree of the polynomial g
    a: (np.array of size deg_true) parameter of g
    N: (int) size of sample to draw

    Returns:
    x: (np.array of size N)
    y: (np.array of size N)
    """
    x = np.sort(np.random.rand(N))
    X = get_design_mat(x, deg_true)
    y = X @ a
    return x, y


def draw_sample_with_noise(deg_true, a, N):
    """
    Inputs:
    deg_true: (int) degree of the polynomial g
    a: (np.array of size deg_true) parameter of g
    N: (int) size of sample to draw

    Returns:
    x: (np.array of size N)
    y: (np.array of size N)
    """
    x = np.random.rand(N)
    X = get_design_mat(x, deg_true)
    y = X @ a + np.random.randn(N)
    return x, y

# Define our least squares estimator function


def least_squares_estimator(X, y):
    """
    Inputs:
    X: (np.matrix of size N x (deg_true +1))
    y: (np.array) of size deg_true + 1 x 1

    Returns:
    b_hat: (np.array) of size N x (deg_true + 1)
    """
    # Make sure N > d
    if X.shape[0] < X.shape[1]:
        raise ValueError("You must have at least as many rows as columns!")
    else:
        # Compute the solution for b using the closed form linear algebra solution
        b_hat = np.linalg.inv(X.T@X) @ X.T @ y
        return b_hat


def empirical_risk(X, y, b_hat):
    """
    Inputs:
    X: (np.matrix of size N x (deg_true +1))
    y: (np.array) of size deg_true + 1 x 1
    b_hat: (np.array) of size N x (deg_true + 1)
    Returns:
    emp_risk: (float) 
    """
    # Get # of observations
    N = X.shape[0]
    # Calculate Predictions
    y_hat = X @ b_hat
    # Calculate squared errors and then empirical risk
    sum_of_squared_errors = sum((y_hat-y)**2)
    emp_risk = sum_of_squared_errors / N
    emp_risk = emp_risk / 2  # because we have 1/2 in our loss function
    return emp_risk


def noisy_emp_and_gen_risk(d, x_train, y_train, x_test, y_test):
    """
    Inputs:
    d: (int) degree of polynomial desired
    n: (int) number of samples to be generated in 

    Outputs:
    training_error: (float) average sum of squares of loss function on training data
    generalization_error: (float) average Sum of Squares of loss function on test data
    """
    # Generate design matrices
    X_train = get_design_mat(x_train, d)
    X_test = get_design_mat(x_test, d)

    # Calculate b_hat
    b_hat = least_squares_estimator(X_train, y_train)
    training_error = empirical_risk(X_train, y_train, b_hat)
    generalization_error = empirical_risk(X_test, y_test, b_hat)

    return training_error, generalization_error, b_hat


def poly_risk_gen(d, n, x_train, y_train, x_test, y_test):
    """
    Ouptut:
    train_error_arr: (np.array) e_t for various n
    test_error_arr: (np.array) e_g for various n
    """
    # Initiliaze np arrays
    train_error_arr = []
    test_error_arr = []

    # Iterate over N
    for i in range(d+1, len(n)):

        # Get relevant subset of data
        train_x_subset = x_train[:i+1]
        train_y_subset = y_train[:i+1]

        # Calculate e_t, e_g, and append to output
        training_error, generalization_error, b_hat = noisy_emp_and_gen_risk(
            d, train_x_subset, train_y_subset, x_test, y_test)
        train_error_arr.append(training_error)
        test_error_arr.append(generalization_error)

    return train_error_arr, test_error_arr, b_hat


In [52]:
# Set the degree of the polynomial
d = 3

# Return coefficients
coef = get_d(d)

# Generate training and test data -> N_train is the size of the train sample to draw, N_test for test
N_train = 100
N_test = 1000

X_train, y_train = draw_sample(d, coef, N_train)
X_test, y_test = draw_sample(d, coef, N_test)

# Generate design matrices
Xd_train = get_design_mat(X_train, d)
Xd_test = get_design_mat(X_test, d)

b_hat = least_squares_estimator(Xd_train, y_train)

# Compare coef and b_hat values (should be same as we provide the data-generating distribution)
for i in range(len(b_hat)):
    print('Values at Index', i,
          'For Vectors b_Hat and coef', coef[i], b_hat[i])
    print("Difference rounded to 5 decimal places:",
          np.round(coef[i]-b_hat[i], 5))


Values at Index 0 For Vectors b_Hat and coef 1.2401199240321432 1.2401199240328558
Difference rounded to 5 decimal places: -0.0
Values at Index 1 For Vectors b_Hat and coef 4.660482295807032 4.660482295806109
Difference rounded to 5 decimal places: 0.0
Values at Index 2 For Vectors b_Hat and coef -3.60348096540093 -3.603480965401607
Difference rounded to 5 decimal places: 0.0
Values at Index 3 For Vectors b_Hat and coef 3.0169141328419995 3.016914132842649
Difference rounded to 5 decimal places: -0.0


In [53]:
# Iterate through values 1-> 7, which will be
# the degree of our polynomial used to predict y (lowest risk should be from coeff degree chosen)
for i in range(1, 7):

    # Get new design matrices for polynomial i
    Xd_train = get_design_mat(X_train, i)
    Xd_test = get_design_mat(X_test, i)

    # Calculate best coefficient for each term
    new_b_hat = least_squares_estimator(Xd_train, y_train)

    # Calculate empirical risk for polynomial degree i
    # Use the test set that we generated
    current_risk = empirical_risk(Xd_test, y_test, new_b_hat)
    print("Empirical Risk for Polynomial Degree", i, "is:", current_risk)


Empirical Risk for Polynomial Degree 1 is: 0.004491582698944798
Empirical Risk for Polynomial Degree 2 is: 0.0018319961209322514
Empirical Risk for Polynomial Degree 3 is: 6.005538633243545e-26
Empirical Risk for Polynomial Degree 4 is: 9.935065881852637e-25
Empirical Risk for Polynomial Degree 5 is: 4.624649950054569e-21
Empirical Risk for Polynomial Degree 6 is: 1.1032789503415328e-18
